In [6]:
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
import pandas as pd
import torch

# 1. Load your data
df = pd.read_csv("../data/4c_stage1_reranker_4b_withsimple_data.csv")

In [7]:
# 2. Convert to sentence-transformers InputExample format
train_samples = [
    InputExample(texts=[row['query'], row['label']], label=float(row['score']))
    for _, row in df.iterrows()
]

In [8]:
# 3. Initialize CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", num_labels=1)

In [9]:
# 4. Define callback to print loss every step
def print_loss(score, step):
    print(f"Step {step} - Loss: {score:.4f}")

# 5. Train the model (fit handles batching, loss, optimizer, etc.)
model.fit(
    train_dataloader=DataLoader(train_samples, shuffle=True, batch_size=16),
    epochs=8,
    warmup_steps=100,
    show_progress_bar=False,
    callback=print_loss
)

{'loss': 0.1768, 'grad_norm': 1.93841552734375, 'learning_rate': 6.04895104895105e-06, 'epoch': 5.882352941176471}
{'train_runtime': 73.117, 'train_samples_per_second': 148.474, 'train_steps_per_second': 9.3, 'train_loss': 0.13067659135250484, 'epoch': 8.0}


In [10]:
model.save('../models/stage1_cross_encoder_finetuned_MiniLM_noisyhardnegative_v4_withcleandata')